# import data

In [1]:
import pandas as pd
import os

stk_code = '603355'
exchange = 'sh_exchange'
folder_path = f'/Users/bokie/code/python/CMS/TWAP优化/trading-optimize/stock_data/{exchange}/{stk_code}'

files_pkl = os.listdir(folder_path)
files_pkl.remove('.DS_Store')
files_pkl
stk_dic = {}
for file in files_pkl:
    tmp_path = folder_path + '/' + file
    df_tmp = pd.read_pickle(tmp_path)
    tmp_key = file[7:15]
    stk_dic[tmp_key] = df_tmp
# date = '20220812'
# date = '20220815'
# date = '20220816'
# date = '20220817'
# date = '20220819'
date = '20220919'

date_formatted = date[:4] + '-' + date[4:6] + '-' + date[6:]
# pd.set_option('display.max_rows', None)
# stk_dic[date]['index_col'] = stk_dic[date]['date_time']
# row.set_index('index_col').iloc[:,113:114]    


In [2]:
date[:4] + '-' + date[4:6] + '-' + date[6:]

'2022-09-19'

In [3]:
full_cols = stk_dic[date].columns.tolist()
print(full_cols)

['auto_id', 'task_id', 'file_id', 'security_id', 'date_time', 'pre_close_px', 'open_px', 'high_px', 'low_px', 'last_px', 'total_volume_trade', 'total_amount_trade', 'instrument_status', 'bid_price1', 'bid_price2', 'bid_price3', 'bid_price4', 'bid_price5', 'bid_price6', 'bid_price7', 'bid_price8', 'bid_price9', 'bid_price10', 'bid_order_qty1', 'bid_order_qty2', 'bid_order_qty3', 'bid_order_qty4', 'bid_order_qty5', 'bid_order_qty6', 'bid_order_qty7', 'bid_order_qty8', 'bid_order_qty9', 'bid_order_qty10', 'bid_num_orders1', 'bid_num_orders2', 'bid_num_orders3', 'bid_num_orders4', 'bid_num_orders5', 'bid_num_orders6', 'bid_num_orders7', 'bid_num_orders8', 'bid_num_orders9', 'bid_num_orders10', 'bid_orders1', 'bid_orders2', 'bid_orders3', 'bid_orders4', 'bid_orders5', 'bid_orders6', 'bid_orders7', 'bid_orders8', 'bid_orders9', 'bid_orders10', 'bid_orders11', 'bid_orders12', 'bid_orders13', 'bid_orders14', 'bid_orders15', 'bid_orders16', 'bid_orders17', 'bid_orders18', 'bid_orders19', 'bid_o

In [4]:
stk_dic.keys()

dict_keys(['20220812', '20220609', '20220621', '20220620', '20220608', '20220805', '20220811', '20220622', '20220623', '20220810', '20220804', '20220627', '20220815', '20220801', '20220829', '20220817', '20220803', '20220630', '20220624', '20220802', '20220816', '20220905', '20220722', '20220906', '20220721', '20220720', '20220708', '20220907', '20220913', '20220718', '20220719', '20220725', '20220902', '20220916', '20220914', '20220727', '20220726', '20220915', '20220901', '20220919', '20220728', '20220714', '20220701', '20220715', '20220729', '20220922', '20220711', '20220705', '20220704', '20220923', '20220909', '20220921', '20220706', '20220712', '20220713', '20220707', '20220920', '20220908', '20220628', '20220614', '20220601', '20220615', '20220629', '20220826', '20220824', '20220830', '20220818', '20220617', '20220616', '20220602', '20220819', '20220831', '20220825', '20220809', '20220606', '20220613', '20220607', '20220808', '20220822', '20220610', '20220823'])

# signal calculate

In [5]:
pd.set_option('display.max_rows', None)
cols = ['date_time', 'bid_price1','offer_price1','bid_order_qty1','offer_order_qty1','last_px']
stk_dic[date][cols].head()

,date_time,bid_price1,offer_price1,bid_order_qty1,offer_order_qty1,last_px
0,20220919091507,30.01,30.01,100,100,0.0
1,20220919091510,31.99,31.99,1400,1400,0.0
2,20220919091622,31.99,31.99,1400,1400,0.0
3,20220919091752,31.99,31.99,1400,1400,0.0
4,20220919091813,31.99,31.99,1400,1400,0.0


In [6]:
import numpy as np
df_stk_indicator = pd.DataFrame(data=[],columns=['wt','vt'])
df_stk_indicator
pre_bid = stk_dic[date]['bid_price1'].tolist()[0]
pre_offer = stk_dic[date]['offer_price1'].tolist()[0]
pre_bid_qty = stk_dic[date]['bid_order_qty1'].tolist()[0]
pre_offer_qty = stk_dic[date]['offer_order_qty1'].tolist()[0]

for index,row in stk_dic[date][cols].set_index('date_time').iterrows():

    if row['bid_price1'] > pre_bid:
        tmp_wt = np.log(row['bid_order_qty1'])
        
    elif row['bid_price1'] == pre_bid:
        tmp_wt = np.log(row['bid_order_qty1']) - np.log(pre_bid_qty)
    elif row['bid_price1'] < pre_bid:
        tmp_wt = -np.log(pre_bid_qty)
    
    if row['offer_price1'] > pre_offer:
        tmp_vt = -np.log(pre_offer_qty)
    elif row['offer_price1'] == pre_offer:
        tmp_vt = np.log(row['offer_order_qty1']) - np.log(pre_offer_qty)
    elif row['offer_price1'] < pre_offer:
        tmp_vt = np.log(row['offer_order_qty1'])

    df_stk_indicator.loc[pd.to_datetime(str(index)),'wt'] = tmp_wt #原始的时间标签是int类型，先转成str再转datetime
    df_stk_indicator.loc[pd.to_datetime(str(index)),'vt'] = tmp_vt
    df_stk_indicator.loc[pd.to_datetime(str(index)),'price'] = row['last_px']
    pre_bid = row['bid_price1']
    pre_offer = row['offer_price1']
    pre_bid_qty = row['bid_order_qty1']
    pre_offer_qty = row['offer_order_qty1']
df_stk_indicator['log-OFI'] = df_stk_indicator['wt'] - df_stk_indicator['vt']

/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_63676/1751040940.py:24: RuntimeWarning: divide by zero encountered in log
  tmp_vt = np.log(row['offer_order_qty1'])
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_63676/1751040940.py:20: RuntimeWarning: divide by zero encountered in log
  tmp_vt = -np.log(pre_offer_qty)


In [7]:
# df_stk_indicator.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']['log-OFI'].apply(float).describe()

In [8]:
df_stk_indicator.head(100)

,wt,vt,price,log-OFI
2022-09-19 09:15:07,0.0,0.0,0.00,0.0
2022-09-19 09:15:10,7.244228,-4.60517,0.00,11.849398
2022-09-19 09:16:22,0.0,0.0,0.00,0.0
2022-09-19 09:17:52,0.0,0.0,0.00,0.0
2022-09-19 09:18:13,0.0,0.0,0.00,0.0
2022-09-19 09:18:22,0.0,0.0,0.00,0.0
2022-09-19 09:18:31,0.0,0.0,0.00,0.0
2022-09-19 09:18:37,0.0,0.0,0.00,0.0
2022-09-19 09:18:46,1.098612,1.098612,0.00,0.0
2022-09-19 09:19:22,0.0,0.0,0.00,0.0


In [9]:
import plotly.graph_objects as go
 
fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=df_stk_indicator.index, y=df_stk_indicator['log-OFI']),
    
],
)

fig.show()    

In [10]:
df_stk = stk_dic[date].copy(deep=True)
df_stk['date_time'] = df_stk['date_time'].apply(lambda x:pd.to_datetime(str(x)))
df_stk = df_stk.set_index('date_time')

In [11]:
f'{date_formatted} 09:30:00'

'2022-09-19 09:30:00'

In [12]:
# df_stk_indicator.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']

In [13]:
df_signal = df_stk_indicator.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']
df_stk = df_stk.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']

df_position = pd.DataFrame(data=[],columns=['pos'])
sigma = df_stk_indicator.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['std']
mean = df_stk_indicator.loc[f'{date_formatted} 09:30:00':f'{date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['mean']
# short_threshold = mean + 2*sigma
# long_threshold = mean - 2*sigma
short_threshold = 10
long_threshold = -10
for index,row in df_signal.iterrows():
    if row['log-OFI'] > short_threshold:
        df_position.loc[index,'pos'] = 1
    elif row['log-OFI'] < long_threshold:
        df_position.loc[index,'pos'] = -1
    else:
        df_position.loc[index,'pos'] = 0
    df_position.loc[index,'price'] = row['price']
df_position['log-OFI'] = df_signal['log-OFI']
# df_position['price'] = df_stk['last_px']
df_position.head(20)

,pos,price,log-OFI
2022-09-19 09:30:00,0,31.32,7.223608
2022-09-19 09:30:03,0,30.74,-4.242265
2022-09-19 09:30:06,0,30.99,-1.791759
2022-09-19 09:30:09,0,30.80,-9.21034
2022-09-19 09:30:12,0,31.28,5.703782
2022-09-19 09:30:15,1,30.82,11.512925
2022-09-19 09:30:18,-1,30.82,-13.955273
2022-09-19 09:30:21,0,30.82,-7.244228
2022-09-19 09:30:24,1,30.87,14.739769
2022-09-19 09:30:27,0,30.80,-9.749753


# backtest

In [14]:
df_trade = pd.DataFrame([])


cash = 0
asset_value = 0

last_cash = 0
last_pos = 0
stock_hold = 0

commission_rate = 2/10000
tax_rate = 1/1000

# commission_rate = 0
# tax_rate = 0

for index,row in df_position.iterrows():
    stk_price = row['price']
    pos = row['pos']
    if pos != last_pos and last_pos==0:
        if pos == -1:
            cash += stk_price 
            cash = cash - (commission_rate + tax_rate)*stk_price #从空仓到short 佣金+印花税
            stock_hold = -1
        elif pos == 1:
            cash -= stk_price
            cash = cash - commission_rate*stk_price #从空仓到long 佣金
            stock_hold = 1
    elif pos != last_pos and last_pos==1:
        if pos == -1:
            cash += stk_price*2
            cash  = cash - (commission_rate + tax_rate)*stk_price*2 #从long到short 佣金+印花税double
            stock_hold = -1
        elif pos == 0:
            cash += stk_price
            cash = cash - (commission_rate + tax_rate)*stk_price #从long到空仓 佣金+印花税
            stock_hold = 0
    elif pos != last_pos and last_pos==-1:
        if pos == 1:
            cash -= stk_price*2
            cash = cash - commission_rate*stk_price*2 #从short到long 佣金double
            stock_hold = 1
        elif pos == 0:
            cash -= stk_price
            cash = cash - commission_rate*stk_price #从short到空仓 佣金
            stock_hold = 0
    asset_value = stock_hold*stk_price
    last_pos = pos
    # print(row.keys())
    df_trade.loc[index,'log-OFI'] = row['log-OFI']
    df_trade.loc[index,'pos'] = pos
    df_trade.loc[index,'stock hold'] = stock_hold
    df_trade.loc[index,'price'] = stk_price
    df_trade.loc[index,'cash'] = cash
    df_trade.loc[index,'asset value'] = asset_value
    df_trade.loc[index,'net value'] = cash + asset_value
    
df_trade['long/short threshold'] = [str(long_threshold)+'@'+str(short_threshold)]*len(df_trade)


In [15]:
# df_trade

# performance analysis

In [16]:
 
fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=df_trade['net value'].index, y=df_trade['net value']),
    
],
)

fig.show() 